In [ ]:
import numpy as np
import pandas as pd
import tensorflow as tf
import matplotlib.pyplot as plt
import seaborn as sns
import random
import scipy.stats
from itertools import islice
from IPython.display import display

from sklearn.model_selection import train_test_split
from sklearn.metrics import f1_score, accuracy_score

from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Flatten
from keras.utils import to_categorical 

import warnings
warnings.filterwarnings('ignore')
import os
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3' 

In [ ]:
from sklearn.datasets import fetch_openml
mnist = fetch_openml(name='mnist_784')

In [ ]:
all_images = mnist.data.values
all_labels = np.array(list(map(int, mnist.target.values)))
all_images = all_images / 255.0

### Create training and test sets

In [ ]:
# Split dataset into training and validation sets
X_train, X_test, y_train, y_test = train_test_split(
    all_images, all_labels, test_size=0.3, shuffle=True, random_state=1337
)

## Create data owners with even distribution

In [ ]:
class DataOwner:
    def __init__(self, data, labels):
        self.data = data
        self.labels = labels
        self.model = None

In [ ]:
# Split the training data into chunks with the provided distribution
distribution = [0.125, 0.125, 0.125, 0.125, 0.125, 0.125, 0.125, 0.125]
data_chunks = chunk_data(X_train, y_train, distribution)
dataOwners = []

# Create dataOwner instances, with data and temporary ML model
for d_c in data_chunks:

    # Give a data owner its portion of data
    new_owner = DataOwner(d_c[0], d_c[1])

    # Create a simple model for each data owner 
    # This will be updated with the weights from global in the future
    new_owner.model = create_simple_model()

    dataOwners.append(new_owner)

## Train data owners individually

In [ ]:
nr_cycles = 0

while nr_cycles < 40:
    for owner in dataOwners:
        fit_model_to_data(owner.model, owner.data, owner.labels, n_classes=10, epochs=2)
    nr_cycles += 1

## Federated Learning Simulation

In [ ]:
# Create global model
global_model = create_simple_model()
global_amount_data = len(X_train)
nr_cycles = 0
while nr_cycles < 40:
    # Extract weights from global model
    global_weights = global_model.get_weights()
    # Simulating sending the global model to the data owners
    for owner in dataOwners: 
        owner_model = owner.model
        owner_model.set_weights(global_weights)
    # Train data owners on their own data
    for owner in dataOwners:
        fit_model_to_data(
            owner.model, 
            owner.data, 
            owner.labels, 
            n_classes=10, 
            epochs=2
        )
    # Simulate the data owners sending their weights to central
    # and calculate data owner weight scaling factor
    owner_weights = []
    scaling_factor_owners = []
    
    for owner in dataOwners:
        weights = np.array(owner.model.get_weights())
        scaling_factor = len(owner.data) / global_amount_data
        
        owner_weights.append(weights)
        scaling_factor_owners.append(scaling_factor)
        
    # Construct new global model from data owner weights
    new_global_weights = np.array(global_weights) * 0 # Create empty weights array 
    for scaling_factor, weights in zip(scaling_factor_owners, owner_weights):
        new_global_weights += weights * scaling_factor  

    # Set the new weights on the global model
    global_model.set_weights(new_global_weights)
    acc = evaluate_model(global_model, X_test, y_test)
    
    nr_cycles += 1
    print("Cycle ", nr_cycles, " complete. Global model accuracy:", acc)

# Helper functions

In [ ]:
def create_simple_model():
    model = Sequential(name="MNIST_Classifier")
    model.add(Dense(784, input_shape = (784,), activation="relu"))
    model.add(Dense(10, activation="softmax"))
    model.compile(
        loss='categorical_crossentropy', 
        optimizer='adam'
    )
    return model

In [ ]:
def fit_model_to_data(model, X, y, n_classes, epochs):
    y_one_hot = to_categorical(y, n_classes)
    model.fit(
        X, # Samples
        y_one_hot, # Labels
        batch_size=32,
        epochs=epochs,
        verbose=0
    )

def evaluate_model(model, X_test, y_test):
    one_hot_predictions = model.predict(X_test)
    label_predictions = np.argmax(one_hot_predictions, axis=1)
    return f1_score(y_test, label_predictions, average='weighted')

In [ ]:
def chunk_array(array, distribution):
    distribution = np.array(distribution) * len(array)
    distribution = [int(d) for d in distribution]
    
    it = iter(array)
    return [np.array(list(islice(it, 0, i))) for i in distribution]

def chunk_data(data, labels, distribution):
    data_chunks = chunk_array(data, distribution)
    label_chunks = chunk_array(labels, distribution)
    return list(zip(data_chunks, label_chunks))

In [ ]:
def plot_data(dataOwners):
    f, axs = plt.subplots(1,2,figsize=(12, 7))
    for i in range(len(dataOwners)):
        labels = dataOwners[i].labels

        plt.subplot(2, 4, i+1)
        plt.title('Owner {}'.format(i+1))
        plt.ylim([0, 4500])
        plt.xlim([-1,10])
        plt.ylabel(" ")
        plt.yticks([])
        plt.xticks(list(np.arange(0,10)))
        sns.histplot(labels, bins=10, discrete=True)
    plt.subplots_adjust(left=0.1,
                    bottom=0.1,
                    right=0.9,
                    top=0.9,
                    wspace=0.1,
                    hspace=0.3)